In [1]:
from datasets import load_dataset
import numpy as np
import time
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
device = "cuda:2" if torch.cuda.is_available() else "cpu"
model = LlamaForCausalLM.from_pretrained(model_path, device_map=device)
tokenizer = LlamaTokenizer.from_pretrained(model_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset = load_dataset('/dataset/crosspipe/Dataset/LongBench', "samsum", split='test')
very_long_context = " ".join(dataset["context"])

PermissionError: [Errno 13] Permission denied: '/dataset/crosspipe/Dataset/LongBench/state.json'

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(very_long_context, max_length=128_000, truncation="max_length", return_tensors="pt").to(model.device)
generation_kwargs = {"do_sample": False, "temperature": 1.0, "top_p": 1.0, "max_new_tokens": 20, "min_new_tokens": 20}
out = model.generate(**inputs, cache_implementation="quantized", cache_config={"backend":"quanto", "nbits": 4, "q_group_size": 64, "residual_length": 128})
# out_fp16 = model.generate(**inputs, **generation_kwargs) # this will crash with OOM

print(f"text with quant cache: {tokenizer.batch_decode(out)}")